In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os.path import join as p_join

import seaborn as sns


##################################
## GLOBAL SETTINGS ###############
##################################
plt.rcParams["figure.figsize"] = (12,8)

In [2]:
### Install CPU torch
# ! pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
    
### Inctall GPU torch (CUDA 11.3)
# ! pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio==0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [3]:
###############################
##### IMPORT DL Depensies #####
###############################
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

from tqdm import tqdm


def seed_all(seed=42):

    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print("[ Using Seed : ", seed, " ]")

####################################
#####   SEED ALL EXPERIMENTS   #####
####################################
seed_all()    


#################################
####### GLOBAL CONFIG ###########
#################################
CONFIG = {'ephs': 5,
          'train_batch_size': 32,
          'test_batch_size': 16,
          'cpu_workers': 2,
          'save_ckpts': True}

[ Using Seed :  42  ]


In [4]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from utils import utils

In [5]:
FOLDERS = [item for item in os.listdir(p_join('..', 'data_2')) if 'L=7.6' in item]
FOLDERS

['N2=21_L=7.6_input=-2',
 'N2=27_L=7.6_input=2',
 'N2=27_L=7.6_input=4',
 'N2=25_L=7.6_input=-2',
 'N2=27_L=7.6_input=-2',
 'N2=23_L=7.6_input=4',
 'N2=25_L=7.6_input=-4',
 'N2=23_L=7.6_input=-2',
 'N2=23_L=7.6_input=2',
 'N2=27_L=7.6_input=-4',
 'N2=23_L=7.6_input=-4',
 'N2=21_L=7.6_input=4',
 'N2=25_L=7.6_input=2',
 'N2=21_L=7.6_input=2',
 'N2=21_L=7.6_input=-4',
 'N2=25_L=7.6_input=4']

### Create dataset

In [6]:
X, Y = utils.create_dataset(FOLDERS)
X = utils.normalize_data(X)
X = np.expand_dims(X, axis=1)

#################################
#### MAKE TEST TRAIN SPLIT   ####
#################################
def create_dataloaders(CONFIG, X, Y):
    from sklearn.model_selection import train_test_split

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42)


    train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
    test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(Y_test))

    train_dataloader = DataLoader(train_dataset,
                                  shuffle=True,
                                  num_workers=CONFIG['cpu_workers'],
                                  batch_size=CONFIG['train_batch_size'],
                                  drop_last=True)

    test_dataloader = DataLoader(test_dataset,
                                 shuffle=False,
                                 num_workers=CONFIG['cpu_workers'],
                                 batch_size=CONFIG['test_batch_size'],
                                 drop_last=False)

    return train_dataloader, test_dataloader

100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 21.84it/s]


### Try more layers

In [7]:
class Block(nn.Module):
    def __init__(self, inp, out, ks):
        super().__init__()
        self.activ = F.gelu
        self.conv1 = nn.Conv1d(inp, out, ks, padding=1)
        self.bn1 = nn.BatchNorm1d(out)

    def forward(self, x):
        x = self.activ(self.bn1(self.conv1(x)))
        return x

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_p = 0.1
        self.h_dim = 240
        self.h_dims = [1, 32, 64, self.h_dim]
        self.backbone = nn.Sequential(*[Block(self.h_dims[i], self.h_dims[i + 1], 3) for i in range(len(self.h_dims) - 1)])
        self.flat = nn.Flatten()
        self.drop = nn.Dropout(p=self.drop_p)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.lin = nn.Sequential(nn.Linear(self.h_dim, self.h_dim * 2),
                                 nn.BatchNorm1d(self.h_dim * 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim * 2, self.h_dim // 2),
                                 nn.BatchNorm1d(self.h_dim // 2),
                                 nn.GELU(),
                                 nn.Dropout(p=self.drop_p),
                                 nn.Linear(self.h_dim // 2, 4))

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = self.flat(x)
        x = self.drop(x)
#         print(x.shape)
        x = self.lin(x)
        return x
    
model = ConvNet()
inp = torch.rand(32, 1, 16)
out = model(inp)
out.shape

CONFIG['train_batch_size'] = 64
CONFIG['ephs'] = 35
net = ConvNet()
train_dataloader, test_dataloader = create_dataloaders(CONFIG, X, Y)
optimizer = torch.optim.Adam(net.parameters(), lr=CONFIG.get('lr', 1e-3))
utils.run_training(net, optimizer, CONFIG, train_dataloader, test_dataloader)

100%|█████████████████████████████████████████| 799/799 [00:31<00:00, 25.17it/s]


Epoch: 1/35, 	 total train loss: 1.0588019384311347


100%|█████████████████████████████████████████| 800/800 [00:09<00:00, 83.02it/s]


Epoch: 1/35, 	 total score test: 51.80901773853247, [best score: 51.80901773853247]



100%|█████████████████████████████████████████| 799/799 [00:42<00:00, 18.78it/s]


...

Epoch: 34/35, 	 total score test: 69.9070094553411, [best score: 70.0945534109557]



100%|█████████████████████████████████████████| 799/799 [01:08<00:00, 11.66it/s]


Epoch: 35/35, 	 total train loss: 0.5291498395170229


100%|█████████████████████████████████████████| 800/800 [00:18<00:00, 43.16it/s]

Epoch: 35/35, 	 total score test: 69.65695084785497, [best score: 70.0945534109557]

